# Memoirr: Preprocessor + Chunker Pipeline Smoke Test

This notebook runs a minimal Haystack pipeline using the SRT preprocessor and the semantic chunker.

Requirements:
- Ensure a local sentence-transformers model is available under `models/<EMBEDDING_MODEL_NAME>/` (with `model.safetensors` and tokenizer/config files).
- `.env` should set `EMBEDDING_MODEL_NAME` (default included in repo). Optionally set `EMBEDDING_DEVICE` (e.g., `cuda:0`).

Notes:
- The preprocessor emits cleaned JSONL lines, one per caption.
- The chunker uses Chonkie SemanticChunker with the self-hosted embeddings to create time-aware chunks.


In [1]:

import pathlib
import textwrap
from src.core.config import get_settings

settings = get_settings()
print('EMBEDDING_MODEL_NAME =', settings.embedding_model_name)
print('EMBEDDING_DEVICE     =', settings.device)

# Quick existence check to help the user
model_path = pathlib.Path('models') / settings.embedding_model_name
if not model_path.exists():
    # Fallback: search by terminal folder name (case-insensitive), similar to runtime resolver
    target = settings.embedding_model_name.split('/')[-1].lower()
    candidates = [p for p in pathlib.Path('models').rglob('*') if p.is_dir() and p.name.lower() == target]
    if candidates:
        print('Found candidate model dir at:', candidates[0])
    else:
        print('WARNING: Expected model folder not found under models/. The chunker cell may fail.')


EMBEDDING_MODEL_NAME = qwen3-embedding-0.6B
EMBEDDING_DEVICE     = 
Found candidate model dir at: models/chunker/qwen3-embedding-0.6B


In [2]:
# Sample SRT content (very small)
sample_srt = textwrap.dedent('''
1
00:00:01,000 --> 00:00:02,000
- Hello there!

2
00:00:02,100 --> 00:00:03,000
How are you doing?

3
00:00:03,100 --> 00:00:04,000
I'm fine. Thanks!
''')
print(sample_srt)



1
00:00:01,000 --> 00:00:02,000
- Hello there!

2
00:00:02,100 --> 00:00:03,000
How are you doing?

3
00:00:03,100 --> 00:00:04,000
I'm fine. Thanks!



In [3]:
# Run the complete end-to-end pipeline: SRT → Preprocess → Chunk → Embed → Qdrant
from src.pipelines.srt_to_qdrant import build_srt_to_qdrant_pipeline

print('Building the complete SRT-to-Qdrant pipeline...')
pipe = build_srt_to_qdrant_pipeline()

print('Pipeline components:')
for component_name in pipe.graph.nodes:
    print(f'  - {component_name}')

print('Pipeline connections:')
for edge in pipe.graph.edges:
    print(f'  - {edge[0]} → {edge[1]}')

print('Running pipeline on sample SRT...')
result = pipe.run({'pre': {'srt_text': sample_srt}})

print('Pipeline Results:')
print('=================')

# Show preprocessing stats
pre_stats = result['pre']['stats']
print(f'Preprocessor: {pre_stats}')

# Show chunking stats
chunk_stats = result['chunk']['stats']
print(f'Chunker: {chunk_stats}')

# Show write stats
write_stats = result['write']['stats']
print(f'Writer: {write_stats}')

print('✅ SUCCESS! Check Qdrant UI at http://localhost:6300/dashboard to see the embedded chunks!')
print('Collection name: memoirr')

2025-09-20T14:53:10.723540Z [info     ] Pipelines module initialized   available_pipelines=['srt_to_qdrant'] environment=development module=pipelines service=memoirr


2025-09-20T14:53:10.943916Z [info     ] Building SRT-to-Qdrant pipeline component=pipeline_builder pipeline_type=srt_to_qdrant


Building the complete SRT-to-Qdrant pipeline...
{'pipeline_type': 'srt_to_qdrant', 'component': 'pipeline_builder', 'event': 'Building SRT-to-Qdrant pipeline', 'level': 'info', 'timestamp': '2025-09-20T14:53:10.943916Z'}


2025-09-20T14:53:14.258216Z [info     ] SRTPreprocessor initialized    component=preprocessor dedupe_window_ms=1000 min_len=1


{'min_len': 1, 'dedupe_window_ms': 1000, 'component': 'preprocessor', 'event': 'SRTPreprocessor initialized', 'level': 'info', 'timestamp': '2025-09-20T14:53:14.258216Z'}


...

{'written_count': 1, 'component': 'qdrant_writer', 'event': 'Documents written successfully', 'level': 'info', 'timestamp': '2025-09-20T14:53:17.633381Z'}


2025-09-20T14:53:17.635153Z [info     ] metric_counter                 component=qdrant_writer metric_name=documents_written_total metric_type=counter status=success value=1


{'metric_name': 'documents_written_total', 'metric_type': 'counter', 'value': 1, 'component': 'qdrant_writer', 'status': 'success', 'event': 'metric_counter', 'level': 'info', 'timestamp': '2025-09-20T14:53:17.635153Z'}


2025-09-20T14:53:17.638352Z [info     ] document_writing_completed     documents_skipped=0 documents_written=1 duration_ms=255 operation=document_writing success_rate=1.0 total_documents=1


{'operation': 'document_writing', 'duration_ms': 255, 'total_documents': 1, 'documents_written': 1, 'documents_skipped': 0, 'success_rate': 1.0, 'event': 'document_writing_completed', 'level': 'info', 'timestamp': '2025-09-20T14:53:17.638352Z'}
Pipeline Results:
Preprocessor: {'total_captions': 3, 'kept': 3, 'dropped_empty': 0, 'dropped_non_english': 0, 'deduped': 0}
Chunker: {'input_captions': 3, 'output_chunks': 1, 'avg_tokens_per_chunk': 15.0}
Writer: {'written': 1, 'skipped': 0, 'total': 1}
✅ SUCCESS! Check Qdrant UI at http://localhost:6300/dashboard to see the embedded chunks!
Collection name: memoirr
